# Scraper

In [2]:
import bs4
import urllib.request

## Le Monde main page

Get the main page in order to get all the article links. 

In [3]:
lemonde = bs4.BeautifulSoup( urllib.request.urlopen("http://www.lemonde.fr/").read(), "lxml")

In [4]:
links = []
for a in lemonde('a'):
    # Avoid subscribed edition and take only article links
    if len(a('', {'class':'marqueur_restreint'})) == 0 and "/article/" in a['href']:
        links.append(a['href'])
len(links)

160

## Article example

Get an article and extract the useful information.

In [5]:
lemonde_article_url = "http://www.lemonde.fr" + links[0]
article_lemonde = bs4.BeautifulSoup( urllib.request.urlopen(lemonde_article_url).read(), "lxml" )
print( "Article: %s" % lemonde_article_url)
print( "Length: %d characters" % len(article_lemonde.text))

Article: http://www.lemonde.fr/europe/article/2017/11/04/carles-puigdemont-sous-le-coup-d-un-mandat-d-arret-europeen_5210043_3214.html
Length: 34886 characters


### Useful information

After a study of what is displayed on an article, I chose to extract those elements:
 - Title
 - Url
 - Article category
 - Writer
 - Article description
 - Publish Time
 - Update Time
 - Article content
 - Related article titles
 
Those elements are not loaded with the main static HTML. It is necessary to make supplementary requests or/and execute javascript.
In a first time, I won't scrape them. As, there is already enough to practice with the previous elements.
 - Comments
     - Writer
     - Date
     - Content
 - \# facebook shares

In [6]:
# Title
if len( article_lemonde('h1', {'class': 'tt2'}) ) > 0:
    title = article_lemonde('h1', {'class': 'tt2'})[0].string
    print ( "* Title: %s\n" % title)

# Simple scrap
if len( article_lemonde('', {'class':'description-article'}) ) > 0:
    article_description = article_lemonde('', {'class':'description-article'})[0].string
    print ( "* Description: \n%s\n" % article_description)

if len( article_lemonde('div', {'id': 'articleBody'}) ) > 0:
    article_content = article_lemonde('div', {'id': 'articleBody'})[0].text.strip()
    limit = len(article_content) if len(article_content)<500 else 500 
    print( "* Article content (cut): \n%s...\n" % article_content[0:500])

related_articles = []
if len( article_lemonde('aside', {'class':'bloc_base meme_sujet'}) ) > 0:
    for art in article_lemonde('aside', {'class':'bloc_base meme_sujet'})[0]('a'):
        related_articles.append(art.text)
    if len( related_articles ) > 0:
        print ( "* First related article title: \n%s" % related_articles[0].strip())

* Title: Catalogne : Puigdemont, sous le coup d’un mandat d’arrêt européen, disposé à être candidat

* Description: 
Le président déchu de l’exécutif catalan a dit, vendredi, qu’il n’avait pas l’intention de fuir la justice et qu’il entendait se présenter aux élections régionales du 21 décembre.

* Article content (cut): 
Le président démis de la Catalogne, Carles Puigdemont, est « disposé à être candidat » aux élections régionales du 21 décembre et à faire campagne depuis l’étranger, a-t-il expliqué, vendredi 3 novembre au soir, sur la chaîne belge RTBF. Au même moment, la justice espagnole lançait des mandats d’arrêt européens à son encontre et celle de quatre de ses anciens ministres régionaux, qui ont fui en Belgique.

        Lire aussi :
         
    
                « L’affaire Puigdemont » empoisonne la ...

* First related article title: 
En Catalogne, les entrepreneurs se préparent « au cas où les choses tourneraient mal »


In [7]:
# Category
offset = len("http://www.lemonde.fr/")-1
slash1 = lemonde_article_url.find('/', offset)+1
slash2 = lemonde_article_url.find('/', slash1)+1
if ( slash1 >= 0 and slash2 >= 0):
    category_1 = lemonde_article_url[slash1:slash2-1]
    print ( "* Category 1: %s" % category_1)

# Writer
if len( article_lemonde('span', {'id': 'publisher'}) ) > 0:
    writer = article_lemonde('span', {'id': 'publisher'})[0].string
    print ( "* Writer: %s" % writer)

# Date
if len( article_lemonde('time', {'itemprop': 'datePublished'}) ) > 0:
    publish_time = article_lemonde('time', {'itemprop': 'datePublished'})[0]['datetime']
    print ( "* Publish time: %s" % publish_time)
if len( article_lemonde('time', {'itemprop': 'dateModified'}) ) > 0:
    update_time = article_lemonde('time', {'itemprop': 'dateModified'})[0]['datetime']
    print ( "* Update time: %s" % update_time)

* Category 1: europe
* Writer: Le Monde
* Publish time: 2017-11-04T06:35:26+01:00
* Update time: 2017-11-04T11:08:58+01:00


### Advanced scrap [Just to see]

Scrap inside of an article content and try to organize it in order to have more information like different titles and paragraphs. This method is more complex and less robust as the HTML references can easily change from article to article.

In a first place, I will only focus on a simple scrap for the ranking system and don't use this one. 

The code below might not return all the information as the HTML references are not enough general for all articles.

In [8]:
# Complete

if len( article_lemonde('h2', {'class':'taille_courante'}) ) > 0:
    article_summary = article_lemonde('h2', {'class':'taille_courante'})[0].text
    print ( "* Summary: \n%s" % article_summary)


print()
article_paragraphs = []
for paragraph in article_lemonde('div', {'id':'articleBody'})[0]('p'):
    if paragraph.get('class') is None:
        article_paragraphs.append(paragraph.text)
print ( "* First paragraph: \n%s" % article_paragraphs[0])

article_intertitres = []
for inter in article_lemonde('h2', {'class':'intertitre'}):
    article_intertitres.append(inter.text)
if len( article_intertitres ) > 0:
    print ( "* First inter title: \n%s" % article_intertitres[0])

* Summary: 
Le président démis de la Catalogne, Carles Puigdemont, est « disposé à être candidat » aux élections régionales du 21 décembre et à faire campagne depuis l’étranger, a-t-il expliqué, vendredi 3 novembre au soir, sur la chaîne belge RTBF. Au même moment, la justice espagnole lançait des mandats d’arrêt européens à son encontre et celle de quatre de ses anciens ministres régionaux, qui ont fui en Belgique.

* First paragraph: 
Pour chacun des prévenus, poursuivis pour « rébellion, sédition, malversation, abus de fonction et désobéissance », la juge d’instruction de l’Audience nationale, le haut tribunal chargé de l’affaire, a émis à la fois un mandat d’arrêt européen adressé au parquet fédéral belge, et un mandat de recherche internationale au cas où M. Puigdemont et ses ex-ministres quitteraient Bruxelles.
* First inter title: 
« Internationaliser » le conflit
